# Recommender System Code Along

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommender').getOrCreate()

In [8]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
data = spark.read.csv('movielens_ratings.csv', inferSchema=True, header=True)

In [5]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
train_data, test_data = data.randomSplit([0.8, 0.2])

In [9]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [10]:
model = als.fit(train_data)

In [12]:
predictions = model.transform(test_data)

In [13]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      2|   4.0|    28|   5.171296|
|      0|   1.0|    27|  0.6046175|
|      0|   1.0|    22|  1.2362825|
|      3|   2.0|    22|  0.6016169|
|      2|   2.0|     1|  0.7887196|
|      0|   1.0|    13| 0.38215062|
|      1|   1.0|     6|  1.4209578|
|      2|   2.0|    20|  1.7112634|
|      1|   1.0|     5|  3.2532833|
|      4|   1.0|    19|  1.3694987|
|      5|   1.0|     9| 0.98765767|
|      6|   1.0|     9|  2.1075442|
|      6|   1.0|     4|  1.3068929|
|      0|   1.0|     8|  1.2147372|
|      3|   2.0|     8|  0.6080357|
|      1|   1.0|     7|   -2.74511|
|      2|   2.0|     7| -1.4033804|
|      0|   3.0|    10|-0.38411504|
|      2|   4.0|    10| 0.57657146|
|      2|   4.0|    21|   2.867773|
+-------+------+------+-----------+
only showing top 20 rows



In [14]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [15]:
rmse = evaluator.evaluate(predictions)

In [16]:
print('RMSE')
print(rmse)

RMSE
1.7763025411429907


In [19]:
single_user = test_data.filter(test_data['userId'] == 11).select(['movieId', 'userId'])

In [20]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     13|    11|
|     41|    11|
|     45|    11|
|     50|    11|
|     51|    11|
|     67|    11|
|     69|    11|
|     78|    11|
|     82|    11|
|     86|    11|
|     88|    11|
+-------+------+



In [22]:
recommendations = model.transform(single_user)

In [23]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-------------+
|movieId|userId|   prediction|
+-------+------+-------------+
|     82|    11|    4.7476635|
|     50|    11|     3.814326|
|     13|    11|    3.6788166|
|     67|    11|    3.3398325|
|     86|    11|    1.8697246|
|     45|    11|    1.7963432|
|     69|    11|    1.6393224|
|     78|    11|    1.5420284|
|     41|    11|    1.4975536|
|     51|    11|-0.0020876527|
|     88|    11|   -0.4403187|
+-------+------+-------------+

